In [1]:
import numpy as np
import pandas as pd
import requests
import json
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt2
import plotly.express as px
import collections
from Functions import *
from IPython.display import display
from datetime import datetime, timedelta
from scipy.stats import norm
from statistics import mean 
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

## Importing the Data

In [3]:
loc_df = pd.read_csv('../data/loc_export_public_vehicle_events_2025_06_10_13_00_33_385.csv')
vehicle_2_loc_df = pd.read_csv('../data/vehicle2/loc_spare_1947_export_public_vehicle_events_2025_06_18_10_07_45_219.csv', index_col=False)
vehicle_2_loc_df

,device_id,id,vehicle_event_type_id,vehicle_id,data,date_created
0,2,18635,0,1947,"{\n 'x': -1139834680,\n 'y': 509375198,\n '...",'2025-04-10T22:31:21.870726+00:00'
1,3,439609,0,1947,"{\n 'x': -1141323550,\n 'y': 510477683,\n '...",'2025-05-04T02:44:28+00:00'
2,3,439610,0,1947,"{\n 'x': -1141323550,\n 'y': 510477683,\n '...",'2025-05-04T02:58:39+00:00'
3,3,439612,0,1947,"{\n 'x': -1141323550,\n 'y': 510477683,\n '...",'2025-05-04T02:58:39.01+00:00'
4,3,439614,0,1947,"{\n 'x': -1141323883,\n 'y': 510476866,\n '...",'2025-05-04T02:58:40+00:00'
...,...,...,...,...,...,...
31338,3,3422009,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",'2025-06-18T12:00:59+00:00'
31339,3,3422020,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",'2025-06-18T13:00:59+00:00'
31340,3,3428531,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",'2025-06-18T14:00:59+00:00'
31341,3,3455829,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",'2025-06-18T15:00:59+00:00'


In [4]:
trip_df = pd.read_csv('../data/trip_export_public_vehicle_events_2025_06_10_13_00_24_215.csv')
vehicle_2_trip_df = pd.read_csv('../data/vehicle2/trip_spare_1947_export_public_vehicle_events_2025_06_18_10_08_56_106.csv', index_col=False)
vehicle_2_trip_df

,device_id,id,vehicle_event_type_id,vehicle_id,data,date_created
0,3,439632,250,1947,1,'2025-05-04T02:59:08+00:00'
1,3,440964,250,1947,0,'2025-05-04T03:12:50+00:00'
2,3,441313,250,1947,1,'2025-05-04T19:58:50+00:00'
3,3,444510,250,1947,0,'2025-05-04T20:18:34+00:00'
4,3,464044,250,1947,1,'2025-05-04T22:50:03+00:00'
...,...,...,...,...,...,...
178,3,3189173,250,1947,0,'2025-06-13T21:43:02+00:00'
179,3,3205534,250,1947,1,'2025-06-13T23:13:17+00:00'
180,3,3206055,250,1947,0,'2025-06-13T23:17:41+00:00'
181,3,3206146,250,1947,1,'2025-06-13T23:22:23+00:00'


In [5]:
trip_df2 = trip_df
loc_df2 = loc_df
vehicle_2_loc_df2 = vehicle_2_loc_df
vehicle_2_trip_df2 = vehicle_2_trip_df

In [6]:
# Convert date columns
trip_df2 = convert_to_datetime(trip_df2, 'date_created')
loc_df2 = convert_to_datetime(loc_df2, 'date_created')

vehicle_2_trip_df2 = convert_to_datetime(vehicle_2_trip_df2, 'date_created')
vehicle_2_loc_df2 = convert_to_datetime(vehicle_2_loc_df2, 'date_created')

In [7]:
# remove fake data | device_id ==2 is fake data
loc_df2 = loc_df2[loc_df2['device_id']==1]
loc_df2 = loc_df2.reset_index(drop=True)

### Extract Loc data

In [9]:
data_str = loc_df2['data'][0]
data_str

"{\n  'x': -1140284616,\n  'y': 510955433,\n  'angle': 121,\n  'speed': 77,\n  'altitude': 1081,\n  'satellites': 19\n}"

In [10]:
loc_df2[['x', 'y']] = loc_df2['data'].apply(extract_coords)
vehicle_2_loc_df2[['x', 'y']] = vehicle_2_loc_df2['data'].apply(extract_coords)
vehicle_2_loc_df2

,device_id,id,vehicle_event_type_id,vehicle_id,data,date_created,x,y
0,2,18635,0,1947,"{\n 'x': -1139834680,\n 'y': 509375198,\n '...",2025-04-10 22:31:21.870726+00:00,-1139834680,509375198
1,3,439609,0,1947,"{\n 'x': -1141323550,\n 'y': 510477683,\n '...",2025-05-04 02:44:28+00:00,-1141323550,510477683
2,3,439610,0,1947,"{\n 'x': -1141323550,\n 'y': 510477683,\n '...",2025-05-04 02:58:39+00:00,-1141323550,510477683
3,3,439612,0,1947,"{\n 'x': -1141323550,\n 'y': 510477683,\n '...",2025-05-04 02:58:39.010000+00:00,-1141323550,510477683
4,3,439614,0,1947,"{\n 'x': -1141323883,\n 'y': 510476866,\n '...",2025-05-04 02:58:40+00:00,-1141323883,510476866
...,...,...,...,...,...,...,...,...
31338,3,3422009,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",2025-06-18 12:00:59+00:00,-1141746966,510379166
31339,3,3422020,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",2025-06-18 13:00:59+00:00,-1141746966,510379166
31340,3,3428531,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",2025-06-18 14:00:59+00:00,-1141746966,510379166
31341,3,3455829,0,1947,"{\n 'x': -1141746966,\n 'y': 510379166,\n '...",2025-06-18 15:00:59+00:00,-1141746966,510379166


### Sort trip by device ID and date created (trip)

In [12]:
trip_df2 = sort_dataframe(trip_df2, 'date_created', ascending=True, reset_idx=True)
vehicle_2_trip_df2 = sort_dataframe(vehicle_2_trip_df2, 'date_created', ascending=True, reset_idx=True)
vehicle_2_trip_df2

,device_id,id,vehicle_event_type_id,vehicle_id,data,date_created
0,3,439632,250,1947,1,2025-05-04 02:59:08+00:00
1,3,440964,250,1947,0,2025-05-04 03:12:50+00:00
2,3,441313,250,1947,1,2025-05-04 19:58:50+00:00
3,3,444510,250,1947,0,2025-05-04 20:18:34+00:00
4,3,464044,250,1947,1,2025-05-04 22:50:03+00:00
...,...,...,...,...,...,...
178,3,3189173,250,1947,0,2025-06-13 21:43:02+00:00
179,3,3205534,250,1947,1,2025-06-13 23:13:17+00:00
180,3,3206055,250,1947,0,2025-06-13 23:17:41+00:00
181,3,3206146,250,1947,1,2025-06-13 23:22:23+00:00


### Remove redundant trips

In [14]:
# Check  the rows where the value in the data column changes from the previous row
trip_df3 = filter_changes(trip_df2, 'data')
vehicle_2_trip_df3 = filter_changes(vehicle_2_trip_df2, 'data')

In [15]:
# Reset index
trip_df3 = trip_df3.reset_index(drop=True)
vehicle_2_trip_df3 = vehicle_2_trip_df3.reset_index(drop=True)

In [16]:
trip_starts = filter_trip_events(trip_df3, column_name='data', event_value=1)
trip_ends = filter_trip_events(trip_df3, column_name='data', event_value=0)
vehicle_2_trip_starts = filter_trip_events(vehicle_2_trip_df3, column_name='data', event_value=1)
vehicle_2_trip_ends = filter_trip_events(vehicle_2_trip_df3, column_name='data', event_value=0)

In [17]:
trip_starts.reset_index(drop=True, inplace=True)
trip_ends.reset_index(drop=True, inplace=True)
vehicle_2_trip_starts.reset_index(drop=True, inplace=True)
vehicle_2_trip_ends.reset_index(drop=True, inplace=True)
vehicle_2_trip_starts

,device_id,id,vehicle_event_type_id,vehicle_id,data,date_created
0,3,439632,250,1947,1,2025-05-04 02:59:08+00:00
1,3,441313,250,1947,1,2025-05-04 19:58:50+00:00
2,3,464044,250,1947,1,2025-05-04 22:50:03+00:00
3,3,563283,250,1947,1,2025-05-06 15:49:15.010000+00:00
4,3,563620,250,1947,1,2025-05-06 18:22:07.010000+00:00
...,...,...,...,...,...,...
86,3,3164768,250,1947,1,2025-06-13 19:59:03.010000+00:00
87,3,3181897,250,1947,1,2025-06-13 20:37:01+00:00
88,3,3187639,250,1947,1,2025-06-13 21:38:09+00:00
89,3,3205534,250,1947,1,2025-06-13 23:13:17+00:00


# ///////////////////////////////////////////////////////////////////////////

In [19]:
# Sort based on trip starts
trip_starts = sort_dataframe(trip_starts, 'date_created', ascending=True, reset_idx=False)
trip_ends = sort_dataframe(trip_ends, 'date_created', ascending=True, reset_idx=False)
vehicle_2_trip_starts = sort_dataframe(vehicle_2_trip_starts, 'date_created', ascending=True, reset_idx=False)
vehicle_2_trip_ends = sort_dataframe(vehicle_2_trip_ends, 'date_created', ascending=True, reset_idx=False)

In [20]:
# Only keep a row if it's the first one, or different from the previous one
trip_starts_2 = filter_changes(trip_starts, 'date_created')
trip_ends_2 = filter_changes(trip_ends, 'date_created')
vehicle_2_trip_starts_2 = filter_changes(vehicle_2_trip_starts, 'date_created')
vehicle_2_trip_ends_2 = filter_changes(vehicle_2_trip_ends, 'date_created')

vehicle_2_trip_starts_2

,device_id,id,vehicle_event_type_id,vehicle_id,data,date_created
0,3,439632,250,1947,1,2025-05-04 02:59:08+00:00
1,3,441313,250,1947,1,2025-05-04 19:58:50+00:00
2,3,464044,250,1947,1,2025-05-04 22:50:03+00:00
3,3,563283,250,1947,1,2025-05-06 15:49:15.010000+00:00
4,3,563620,250,1947,1,2025-05-06 18:22:07.010000+00:00
...,...,...,...,...,...,...
86,3,3164768,250,1947,1,2025-06-13 19:59:03.010000+00:00
87,3,3181897,250,1947,1,2025-06-13 20:37:01+00:00
88,3,3187639,250,1947,1,2025-06-13 21:38:09+00:00
89,3,3205534,250,1947,1,2025-06-13 23:13:17+00:00


# ///////////////////////////////////////////////////////////////////////////

In [22]:
# Pair starts and stops as one trip
paired_trips = pair_trip_events(trip_starts, trip_ends, id_columns=['device_id', 'vehicle_id'],
                     start_col='date_created', end_col='date_created')
vehicle_2_paired_trips = pair_trip_events(vehicle_2_trip_starts, vehicle_2_trip_ends, id_columns=['device_id', 'vehicle_id'],
                     start_col='date_created', end_col='date_created')

In [23]:
paired_trips2 = paired_trips
vehicle_2_paired_trips_2 = vehicle_2_paired_trips

In [24]:
paired_trips2 = paired_trips2.dropna()
vehicle_2_paired_trips_2 = vehicle_2_paired_trips_2.dropna()
vehicle_2_paired_trips_2

,device_id,vehicle_id,start_time,end_time
0,3,1947,2025-05-04 02:59:08.000,2025-05-04 03:12:50.000
1,3,1947,2025-05-04 19:58:50.000,2025-05-04 20:18:34.000
2,3,1947,2025-05-04 22:50:03.000,2025-05-04 23:07:37.000
3,3,1947,2025-05-06 15:49:15.010,2025-05-06 16:03:24.010
4,3,1947,2025-05-06 18:22:07.010,2025-05-06 18:33:35.000
...,...,...,...,...
86,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000
87,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000
88,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000
89,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000


In [25]:
# Save to csv
paired_trips2.to_csv('../data/paired_trips_2_clean.csv.csv', index=False)
vehicle_2_paired_trips_2.to_csv('../data/vehicle2/vehicle_2_paired_trips_2_clean.csv', index=False)

In [26]:
paired_trips2 = pd.read_csv('../data/paired_trips_2_clean.csv')
vehicle_2_paired_trips_2 = pd.read_csv('../data/vehicle2/vehicle_2_paired_trips_2_clean.csv')

In [27]:
# Convert Columns to Datetime
paired_trips2 = convert_to_datetime(paired_trips2, 'start_time')
paired_trips2 = convert_to_datetime(paired_trips2, 'end_time')
vehicle_2_paired_trips_2 = convert_to_datetime(vehicle_2_paired_trips_2, 'start_time')
vehicle_2_paired_trips_2 = convert_to_datetime(vehicle_2_paired_trips_2, 'end_time')

vehicle_2_paired_trips_2

,device_id,vehicle_id,start_time,end_time
0,3,1947,2025-05-04 02:59:08.000,2025-05-04 03:12:50.000
1,3,1947,2025-05-04 19:58:50.000,2025-05-04 20:18:34.000
2,3,1947,2025-05-04 22:50:03.000,2025-05-04 23:07:37.000
3,3,1947,2025-05-06 15:49:15.010,2025-05-06 16:03:24.010
4,3,1947,2025-05-06 18:22:07.010,2025-05-06 18:33:35.000
...,...,...,...,...
86,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000
87,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000
88,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000
89,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000


In [28]:
# For each start and end time, find closest matching timestamp in location_df
paired_trips2[['start_x', 'start_y', 'start_loc_time']] = paired_trips2.apply(
    lambda row: get_closest_location(row['device_id'], row['start_time'], loc_df2), axis=1
)

paired_trips2[['end_x', 'end_y', 'end_loc_time']] = paired_trips2.apply(
    lambda row: get_closest_location(row['device_id'], row['end_time'], loc_df2), axis=1
)

vehicle_2_paired_trips_2[['start_x', 'start_y', 'start_loc_time']] = vehicle_2_paired_trips_2.apply(
    lambda row: get_closest_location(row['device_id'], row['start_time'], vehicle_2_loc_df2), axis=1
)

vehicle_2_paired_trips_2[['end_x', 'end_y', 'end_loc_time']] = vehicle_2_paired_trips_2.apply(
    lambda row: get_closest_location(row['device_id'], row['end_time'], vehicle_2_loc_df2), axis=1
)

# Result: DataFrame with trips and their start/end coordinates
vehicle_2_paired_trips_2

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time
0,3,1947,2025-05-04 02:59:08.000,2025-05-04 03:12:50.000,-1141325616,510477333,2025-05-04 02:59:08+00:00,-1141747099,510379066,2025-05-04 03:12:50+00:00
1,3,1947,2025-05-04 19:58:50.000,2025-05-04 20:18:34.000,-1141748383,510379016,2025-05-04 19:58:50+00:00,-1141539383,510817433,2025-05-04 20:18:34+00:00
2,3,1947,2025-05-04 22:50:03.000,2025-05-04 23:07:37.000,-1141540583,510819183,2025-05-04 22:50:03+00:00,-1141747016,510378616,2025-05-04 23:07:37+00:00
3,3,1947,2025-05-06 15:49:15.010,2025-05-06 16:03:24.010,-1141747566,510379416,2025-05-06 15:49:15.010000+00:00,-1141232833,510443516,2025-05-06 16:03:24.010000+00:00
4,3,1947,2025-05-06 18:22:07.010,2025-05-06 18:33:35.000,-1141233566,510441816,2025-05-06 18:22:07.010000+00:00,-1141746983,510378649,2025-05-06 18:33:35+00:00
...,...,...,...,...,...,...,...,...,...,...
86,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000,-1141749300,510378633,2025-06-13 19:59:03.010000+00:00,-1141233733,510854683,2025-06-13 20:20:15+00:00
87,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000,-1141234316,510854533,2025-06-13 20:37:01+00:00,-1141614883,510190050,2025-06-13 20:54:24+00:00
88,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000,-1141613566,510189166,2025-06-13 21:38:09+00:00,-1141665850,510214116,2025-06-13 21:43:02+00:00
89,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000,-1141667583,510212550,2025-06-13 23:13:17+00:00,-1141748733,510172150,2025-06-13 23:17:41+00:00


In [29]:
trip_device_ids = set(paired_trips2['device_id'].unique())
location_device_ids = set(loc_df2['device_id'].unique())

vehicle_2_trip_device_ids = set(vehicle_2_paired_trips_2['device_id'].unique())
vehicle_2_location_device_ids = set(vehicle_2_loc_df2['device_id'].unique())

# Check if all device_ids in paired_trips exist in location_df
missing_ids = trip_device_ids - location_device_ids
vehicle_2_missing_ids = vehicle_2_trip_device_ids - vehicle_2_location_device_ids

if not missing_ids:
    print("✅ All device_ids in paired_trips exist in location data.")
else:
    print("❌ These device_ids are in paired_trips but missing in location data:", vehicle_2_missing_ids)

✅ All device_ids in paired_trips exist in location data.


In [30]:
print("paired_trips['start_time']:", paired_trips2['start_time'].dtype)
print("location_df['date_created']:", loc_df2['date_created'].dtype)
print("paired_trips['start_time']:", vehicle_2_paired_trips_2['start_time'].dtype)
print("location_df['date_created']:", vehicle_2_loc_df2['date_created'].dtype)

paired_trips['start_time']: datetime64[ns]
location_df['date_created']: datetime64[ns, UTC]
paired_trips['start_time']: datetime64[ns]
location_df['date_created']: datetime64[ns, UTC]


In [31]:
# Save to csv
paired_trips2.to_csv('../data/NDT_pre_dataset_2.csv', index=False)
vehicle_2_paired_trips_2.to_csv('../data/vehicle2/vehicle_2_paired_trips_2_NDT_pre_dataset_2.csv', index=False)

In [32]:
# Check for violations
paired_trips2[paired_trips2['start_time'] > paired_trips2['end_time']]
vehicle_2_paired_trips_2[vehicle_2_paired_trips_2['start_time'] > vehicle_2_paired_trips_2['end_time']]

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time


### Detecting natural downtime 

In [34]:
paired_trips3 = pd.read_csv('../data/NDT_pre_dataset_2.csv')
vehicle_2_paired_trips_3 = pd.read_csv('../data/vehicle2/vehicle_2_paired_trips_2_NDT_pre_dataset_2.csv')

In [35]:
# Check for violations
paired_trips3[paired_trips3['start_time'] < paired_trips3['end_time']]
vehicle_2_paired_trips_3[vehicle_2_paired_trips_3['start_time'] < vehicle_2_paired_trips_3['end_time']]

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time
0,3,1947,2025-05-04 02:59:08.000,2025-05-04 03:12:50.000,-1141325616,510477333,2025-05-04 02:59:08+00:00,-1141747099,510379066,2025-05-04 03:12:50+00:00
1,3,1947,2025-05-04 19:58:50.000,2025-05-04 20:18:34.000,-1141748383,510379016,2025-05-04 19:58:50+00:00,-1141539383,510817433,2025-05-04 20:18:34+00:00
2,3,1947,2025-05-04 22:50:03.000,2025-05-04 23:07:37.000,-1141540583,510819183,2025-05-04 22:50:03+00:00,-1141747016,510378616,2025-05-04 23:07:37+00:00
3,3,1947,2025-05-06 15:49:15.010,2025-05-06 16:03:24.010,-1141747566,510379416,2025-05-06 15:49:15.010000+00:00,-1141232833,510443516,2025-05-06 16:03:24.010000+00:00
4,3,1947,2025-05-06 18:22:07.010,2025-05-06 18:33:35.000,-1141233566,510441816,2025-05-06 18:22:07.010000+00:00,-1141746983,510378649,2025-05-06 18:33:35+00:00
...,...,...,...,...,...,...,...,...,...,...
86,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000,-1141749300,510378633,2025-06-13 19:59:03.010000+00:00,-1141233733,510854683,2025-06-13 20:20:15+00:00
87,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000,-1141234316,510854533,2025-06-13 20:37:01+00:00,-1141614883,510190050,2025-06-13 20:54:24+00:00
88,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000,-1141613566,510189166,2025-06-13 21:38:09+00:00,-1141665850,510214116,2025-06-13 21:43:02+00:00
89,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000,-1141667583,510212550,2025-06-13 23:13:17+00:00,-1141748733,510172150,2025-06-13 23:17:41+00:00


In [36]:
# Converting to datetime
paired_trips3 = convert_to_datetime(paired_trips3, 'start_time')
paired_trips3 = convert_to_datetime(paired_trips3, 'end_time')
paired_trips3 = convert_to_datetime(paired_trips3, 'start_loc_time')
paired_trips3 = convert_to_datetime(paired_trips3, 'end_loc_time')

vehicle_2_paired_trips_3 = convert_to_datetime(vehicle_2_paired_trips_3, 'start_time')
vehicle_2_paired_trips_3 = convert_to_datetime(vehicle_2_paired_trips_3, 'end_time')
vehicle_2_paired_trips_3 = convert_to_datetime(vehicle_2_paired_trips_3, 'start_loc_time')
vehicle_2_paired_trips_3 = convert_to_datetime(vehicle_2_paired_trips_3, 'end_loc_time')

In [37]:
# Sort data by start_time to ensure proper sequencing
paired_trips4 = sort_dataframe(paired_trips3, 'start_time', ascending=True, reset_idx=True)
vehicle_2_paired_trips_4 = sort_dataframe(vehicle_2_paired_trips_3, 'start_time', ascending=True, reset_idx=True)

vehicle_2_paired_trips_4

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time
0,3,1947,2025-05-04 02:59:08.000,2025-05-04 03:12:50.000,-1141325616,510477333,2025-05-04 02:59:08+00:00,-1141747099,510379066,2025-05-04 03:12:50+00:00
1,3,1947,2025-05-04 19:58:50.000,2025-05-04 20:18:34.000,-1141748383,510379016,2025-05-04 19:58:50+00:00,-1141539383,510817433,2025-05-04 20:18:34+00:00
2,3,1947,2025-05-04 22:50:03.000,2025-05-04 23:07:37.000,-1141540583,510819183,2025-05-04 22:50:03+00:00,-1141747016,510378616,2025-05-04 23:07:37+00:00
3,3,1947,2025-05-06 15:49:15.010,2025-05-06 16:03:24.010,-1141747566,510379416,2025-05-06 15:49:15.010000+00:00,-1141232833,510443516,2025-05-06 16:03:24.010000+00:00
4,3,1947,2025-05-06 18:22:07.010,2025-05-06 18:33:35.000,-1141233566,510441816,2025-05-06 18:22:07.010000+00:00,-1141746983,510378649,2025-05-06 18:33:35+00:00
...,...,...,...,...,...,...,...,...,...,...
86,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000,-1141749300,510378633,2025-06-13 19:59:03.010000+00:00,-1141233733,510854683,2025-06-13 20:20:15+00:00
87,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000,-1141234316,510854533,2025-06-13 20:37:01+00:00,-1141614883,510190050,2025-06-13 20:54:24+00:00
88,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000,-1141613566,510189166,2025-06-13 21:38:09+00:00,-1141665850,510214116,2025-06-13 21:43:02+00:00
89,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000,-1141667583,510212550,2025-06-13 23:13:17+00:00,-1141748733,510172150,2025-06-13 23:17:41+00:00


In [38]:
# Validating that the start times are before end times
paired_trips4[paired_trips4['start_time'] < paired_trips4['end_time']]
vehicle_2_paired_trips_4[vehicle_2_paired_trips_4['start_time'] < vehicle_2_paired_trips_4['end_time']]

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time
0,3,1947,2025-05-04 02:59:08.000,2025-05-04 03:12:50.000,-1141325616,510477333,2025-05-04 02:59:08+00:00,-1141747099,510379066,2025-05-04 03:12:50+00:00
1,3,1947,2025-05-04 19:58:50.000,2025-05-04 20:18:34.000,-1141748383,510379016,2025-05-04 19:58:50+00:00,-1141539383,510817433,2025-05-04 20:18:34+00:00
2,3,1947,2025-05-04 22:50:03.000,2025-05-04 23:07:37.000,-1141540583,510819183,2025-05-04 22:50:03+00:00,-1141747016,510378616,2025-05-04 23:07:37+00:00
3,3,1947,2025-05-06 15:49:15.010,2025-05-06 16:03:24.010,-1141747566,510379416,2025-05-06 15:49:15.010000+00:00,-1141232833,510443516,2025-05-06 16:03:24.010000+00:00
4,3,1947,2025-05-06 18:22:07.010,2025-05-06 18:33:35.000,-1141233566,510441816,2025-05-06 18:22:07.010000+00:00,-1141746983,510378649,2025-05-06 18:33:35+00:00
...,...,...,...,...,...,...,...,...,...,...
86,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000,-1141749300,510378633,2025-06-13 19:59:03.010000+00:00,-1141233733,510854683,2025-06-13 20:20:15+00:00
87,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000,-1141234316,510854533,2025-06-13 20:37:01+00:00,-1141614883,510190050,2025-06-13 20:54:24+00:00
88,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000,-1141613566,510189166,2025-06-13 21:38:09+00:00,-1141665850,510214116,2025-06-13 21:43:02+00:00
89,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000,-1141667583,510212550,2025-06-13 23:13:17+00:00,-1141748733,510172150,2025-06-13 23:17:41+00:00


In [39]:
paired_trips4_clean = paired_trips4[paired_trips4['start_time'] >= '2025-05-08' ].reset_index(drop=True)
vehicle_2_paired_trips4_clean = vehicle_2_paired_trips_4[vehicle_2_paired_trips_4['start_time'] >= '2025-05-08' ].reset_index(drop=True)
vehicle_2_paired_trips4_clean

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time
0,3,1947,2025-05-08 14:17:36.000,2025-05-08 14:39:11.010,-1141748316,510379099,2025-05-08 14:17:36+00:00,-1141313833,510844283,2025-05-08 14:39:11.010000+00:00
1,3,1947,2025-05-08 23:01:11.000,2025-05-08 23:27:35.000,-1141315333,510845216,2025-05-08 23:01:11+00:00,-1141746733,510378983,2025-05-08 23:27:35+00:00
2,3,1947,2025-05-09 19:13:29.000,2025-05-09 19:20:16.010,-1141748016,510379516,2025-05-09 19:13:29+00:00,-1141671116,510206700,2025-05-09 19:20:16.010000+00:00
3,3,1947,2025-05-09 20:12:26.000,2025-05-09 20:16:17.010,-1141671466,510207999,2025-05-09 20:12:26+00:00,-1141736599,510188933,2025-05-09 20:16:17.010000+00:00
4,3,1947,2025-05-09 20:22:19.010,2025-05-09 20:43:31.000,-1141734450,510186400,2025-05-09 20:22:19.010000+00:00,-1141448816,509400083,2025-05-09 20:43:31+00:00
...,...,...,...,...,...,...,...,...,...,...
72,3,1947,2025-06-13 19:59:03.010,2025-06-13 20:20:15.000,-1141749300,510378633,2025-06-13 19:59:03.010000+00:00,-1141233733,510854683,2025-06-13 20:20:15+00:00
73,3,1947,2025-06-13 20:37:01.000,2025-06-13 20:54:24.000,-1141234316,510854533,2025-06-13 20:37:01+00:00,-1141614883,510190050,2025-06-13 20:54:24+00:00
74,3,1947,2025-06-13 21:38:09.000,2025-06-13 21:43:02.000,-1141613566,510189166,2025-06-13 21:38:09+00:00,-1141665850,510214116,2025-06-13 21:43:02+00:00
75,3,1947,2025-06-13 23:13:17.000,2025-06-13 23:17:41.000,-1141667583,510212550,2025-06-13 23:13:17+00:00,-1141748733,510172150,2025-06-13 23:17:41+00:00


In [40]:
# Create downtime records: between end_time[i] and start_time[i+1]
downtime_df = compute_downtime_periods(paired_trips4_clean,
                              start_col='start_time',
                              end_col='end_time',
                              x_col='end_x',
                              y_col='end_y',
                              min_idle_hours=0)

vehicle_2_downtime_df = compute_downtime_periods(vehicle_2_paired_trips4_clean,
                              start_col='start_time',
                              end_col='end_time',
                              x_col='end_x',
                              y_col='end_y',
                              min_idle_hours=0)

vehicle_2_paired_trips4_clean.head()

,device_id,vehicle_id,start_time,end_time,start_x,start_y,start_loc_time,end_x,end_y,end_loc_time
0,3,1947,2025-05-08 14:17:36.000,2025-05-08 14:39:11.010,-1141748316,510379099,2025-05-08 14:17:36+00:00,-1141313833,510844283,2025-05-08 14:39:11.010000+00:00
1,3,1947,2025-05-08 23:01:11.000,2025-05-08 23:27:35.000,-1141315333,510845216,2025-05-08 23:01:11+00:00,-1141746733,510378983,2025-05-08 23:27:35+00:00
2,3,1947,2025-05-09 19:13:29.000,2025-05-09 19:20:16.010,-1141748016,510379516,2025-05-09 19:13:29+00:00,-1141671116,510206700,2025-05-09 19:20:16.010000+00:00
3,3,1947,2025-05-09 20:12:26.000,2025-05-09 20:16:17.010,-1141671466,510207999,2025-05-09 20:12:26+00:00,-1141736599,510188933,2025-05-09 20:16:17.010000+00:00
4,3,1947,2025-05-09 20:22:19.010,2025-05-09 20:43:31.000,-1141734450,510186400,2025-05-09 20:22:19.010000+00:00,-1141448816,509400083,2025-05-09 20:43:31+00:00


In [41]:
# Change from UTC to MDT
downtime_df = convert_timezone(downtime_df, 'idle_start_time', 'America/Edmonton', new_column_name='idle_start_time_mdt')
downtime_df = convert_timezone(downtime_df, 'idle_end_time', 'America/Edmonton', new_column_name='idle_end_time_mdt')
vehicle_2_downtime_df = convert_timezone(vehicle_2_downtime_df, 'idle_start_time', 'America/Edmonton', new_column_name='idle_start_time_mdt')
vehicle_2_downtime_df = convert_timezone(vehicle_2_downtime_df, 'idle_end_time', 'America/Edmonton', new_column_name='idle_end_time_mdt')

In [42]:
# Add hour of day
downtime_df = compute_fractional_hour(downtime_df, 'idle_start_time_mdt', 'idle_end_time_mdt', 'idle_start_hour_mdt')
vehicle_2_downtime_df = compute_fractional_hour(vehicle_2_downtime_df, 'idle_start_time_mdt', 'idle_end_time_mdt', 'idle_start_hour_mdt')

In [43]:
downtime_df_clean = sort_dataframe(downtime_df, 'idle_start_time', ascending=True, reset_idx=True)
vehicle_2_downtime_df_clean = sort_dataframe(vehicle_2_downtime_df, 'idle_start_time', ascending=True, reset_idx=True)
vehicle_2_downtime_df_clean

,idle_start_time,idle_end_time,idle_duration_hr,idle_location_x,idle_location_y,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt
0,2025-05-08 14:39:11.010000+00:00,2025-05-08 23:01:11+00:00,8.366664,-1141313833,510844283,2025-05-08 08:39:11.010000-06:00,2025-05-08 17:01:11-06:00,8.016667
1,2025-05-08 23:27:35+00:00,2025-05-09 19:13:29+00:00,19.765000,-1141746733,510378983,2025-05-08 17:27:35-06:00,2025-05-09 13:13:29-06:00,17.216667
2,2025-05-09 19:20:16.010000+00:00,2025-05-09 20:12:26+00:00,0.869442,-1141671116,510206700,2025-05-09 13:20:16.010000-06:00,2025-05-09 14:12:26-06:00,13.200000
3,2025-05-09 20:16:17.010000+00:00,2025-05-09 20:22:19.010000+00:00,0.100556,-1141736599,510188933,2025-05-09 14:16:17.010000-06:00,2025-05-09 14:22:19.010000-06:00,14.366667
4,2025-05-09 20:43:31+00:00,2025-05-09 20:46:51+00:00,0.055556,-1141448816,509400083,2025-05-09 14:43:31-06:00,2025-05-09 14:46:51-06:00,14.766667
...,...,...,...,...,...,...,...,...
71,2025-06-12 22:32:09.010000+00:00,2025-06-13 19:59:03.010000+00:00,21.448333,-1141746966,510379083,2025-06-12 16:32:09.010000-06:00,2025-06-13 13:59:03.010000-06:00,16.983333
72,2025-06-13 20:20:15+00:00,2025-06-13 20:37:01+00:00,0.279444,-1141233733,510854683,2025-06-13 14:20:15-06:00,2025-06-13 14:37:01-06:00,14.616667
73,2025-06-13 20:54:24+00:00,2025-06-13 21:38:09+00:00,0.729167,-1141614883,510190050,2025-06-13 14:54:24-06:00,2025-06-13 15:38:09-06:00,14.633333
74,2025-06-13 21:43:02+00:00,2025-06-13 23:13:17+00:00,1.504167,-1141665850,510214116,2025-06-13 15:43:02-06:00,2025-06-13 17:13:17-06:00,15.216667


In [44]:
# Save to csv
downtime_df_clean.to_csv('../data/NDT_dataset_clean_2.csv')
vehicle_2_downtime_df_clean.to_csv('../data/vehicle2/vehicle_2_NDT_dataset_clean_2.csv')

In [45]:
downtime_df_clean = downtime_df
vehicle_2_downtime_df_clean = vehicle_2_downtime_df

In [46]:
downtime_df_clean = downtime_df[downtime_df['idle_duration_hr'] > 0].reset_index(drop=True)
vehicle_2_downtime_df_clean = vehicle_2_downtime_df[vehicle_2_downtime_df['idle_duration_hr'] > 0].reset_index(drop=True)
vehicle_2_downtime_df_clean

,idle_start_time,idle_end_time,idle_duration_hr,idle_location_x,idle_location_y,idle_start_time_mdt,idle_end_time_mdt,idle_start_hour_mdt
0,2025-05-08 14:39:11.010000+00:00,2025-05-08 23:01:11+00:00,8.366664,-1141313833,510844283,2025-05-08 08:39:11.010000-06:00,2025-05-08 17:01:11-06:00,8.016667
1,2025-05-08 23:27:35+00:00,2025-05-09 19:13:29+00:00,19.765000,-1141746733,510378983,2025-05-08 17:27:35-06:00,2025-05-09 13:13:29-06:00,17.216667
2,2025-05-09 19:20:16.010000+00:00,2025-05-09 20:12:26+00:00,0.869442,-1141671116,510206700,2025-05-09 13:20:16.010000-06:00,2025-05-09 14:12:26-06:00,13.200000
3,2025-05-09 20:16:17.010000+00:00,2025-05-09 20:22:19.010000+00:00,0.100556,-1141736599,510188933,2025-05-09 14:16:17.010000-06:00,2025-05-09 14:22:19.010000-06:00,14.366667
4,2025-05-09 20:43:31+00:00,2025-05-09 20:46:51+00:00,0.055556,-1141448816,509400083,2025-05-09 14:43:31-06:00,2025-05-09 14:46:51-06:00,14.766667
...,...,...,...,...,...,...,...,...
71,2025-06-12 22:32:09.010000+00:00,2025-06-13 19:59:03.010000+00:00,21.448333,-1141746966,510379083,2025-06-12 16:32:09.010000-06:00,2025-06-13 13:59:03.010000-06:00,16.983333
72,2025-06-13 20:20:15+00:00,2025-06-13 20:37:01+00:00,0.279444,-1141233733,510854683,2025-06-13 14:20:15-06:00,2025-06-13 14:37:01-06:00,14.616667
73,2025-06-13 20:54:24+00:00,2025-06-13 21:38:09+00:00,0.729167,-1141614883,510190050,2025-06-13 14:54:24-06:00,2025-06-13 15:38:09-06:00,14.633333
74,2025-06-13 21:43:02+00:00,2025-06-13 23:13:17+00:00,1.504167,-1141665850,510214116,2025-06-13 15:43:02-06:00,2025-06-13 17:13:17-06:00,15.216667
